In [1]:
import yfinance as yf
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from modules.scalers import GroupByScaler
from sklearn.preprocessing import MaxAbsScaler
import json
import torch

from modules.environment import MultiPeriodPortfolioOptimizationEnv
from modules.architectures import MultiPeriodAttentionNetwork
from modules.models import DRLAgent

sns.set()

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
config_file_name = "config.json"
with open(config_file_name, "r") as jsonfile:
    config_data = json.load(jsonfile)

In [3]:
sp_100 = config_data["tickers"]["America"]["SP100"]
NUM_ASSETS = len(sp_100)
len(sp_100)

101

In [4]:
start_date = config_data["timeframe"]["bull_market"]["start_date"]
end_date = config_data["timeframe"]["bull_market"]["end_date"]
data_interval = config_data["data_interval"]
test_ratio = config_data["train_test_ratio"]
random_seed = config_data["random_state_seed"]

In [5]:
portfolio_raw_df = yf.download(tickers=sp_100, start=start_date, end=end_date, interval=data_interval)

portfolio_raw_df.fillna(method="bfill", inplace=True)
portfolio_raw_df = portfolio_raw_df.stack(level=1).rename_axis(["Date", "Ticker"]).reset_index(level=1)
portfolio_raw_df = portfolio_raw_df.drop("Adj Close", axis=1)
portfolio_raw_df.columns.name = None
portfolio_raw_df = portfolio_raw_df.reset_index()
portfolio_raw_df.Date = portfolio_raw_df.Date.astype(str)
portfolio_raw_df.columns = ["date", "tic", "close", "high", "low", "open", "volume"]
portfolio_raw_df = portfolio_raw_df[["date", "tic", "close", "high", "low", "volume"]]

df_portfolio_raw_train, df_portfolio_raw_test = train_test_split(portfolio_raw_df, test_size=test_ratio, shuffle=False, random_state=random_seed)
df_portfolio_train = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(df_portfolio_raw_train)
df_portfolio_test = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(df_portfolio_raw_test)


[*********************100%%**********************]  101 of 101 completed
/var/folders/02/rmgfx9wj0s99nn4tzfthtrnc0000gn/T/ipykernel_20079/541920902.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  portfolio_raw_df.fillna(method="bfill", inplace=True)
/var/folders/02/rmgfx9wj0s99nn4tzfthtrnc0000gn/T/ipykernel_20079/541920902.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  portfolio_raw_df = portfolio_raw_df.stack(level=1).rename_axis(["Date", "Ticker"]).reset_index(level=1)


In [6]:
# SETTING HYPERPARAMETERS
FEATURES = ["close", "high", "low", "volume"]
N = config_data["lookback_window"]
T = config_data["multi_step_horizon"]
NUM_FEATURES = len(FEATURES)
experiment_type = "TRANSFORMER" 
N, T

(24, 3)

In [7]:
train_environment = MultiPeriodPortfolioOptimizationEnv(
    df_portfolio_train,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=N,
    multi_period_horizon=T,
    features=FEATURES,
    normalize_df=None,
    is_train_mode=True,
    experiment_type=experiment_type
)

In [8]:
# set PolicyGradient parameters
model_kwargs = {
    "lr": 0.001,
    "policy": MultiPeriodAttentionNetwork,
    "multi_period_horizon": T
}

# here, we can set EIIE's parameters
policy_kwargs = {
    "num_features": NUM_ASSETS*NUM_FEATURES,
    "num_stocks": NUM_ASSETS,
    "num_head": 4, # NUM-HEAD needs to be divisible by NUM-FEATURES
    "W": N,
    "T": T
}

In [9]:
model = DRLAgent(train_environment).get_model("pg", device, model_kwargs, policy_kwargs)

In [10]:
print("TRAINING AGENT.....")
DRLAgent.train_model(model, episodes=50)

TRAINING AGENT.....


  0%|          | 0/50 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 1989109.7251272558
Final accumulative portfolio value: 19.89109725127256
Maximum DrawDown: -0.7711704626854992
Sharpe ratio: 5.826286171237248


  2%|▏         | 1/50 [00:08<06:34,  8.06s/it]

Initial portfolio value:100000
Final portfolio value: 1972343.7178246542
Final accumulative portfolio value: 19.72343717824654
Maximum DrawDown: -0.7713724193235216
Sharpe ratio: 5.790137756117682


...

 96%|█████████▌| 48/50 [06:00<00:14,  7.37s/it]

Initial portfolio value:100000
Final portfolio value: 2073785.4691397827
Final accumulative portfolio value: 20.73785469139783
Maximum DrawDown: -0.770364781924638
Sharpe ratio: 5.889340018454481


 98%|█████████▊| 49/50 [06:08<00:07,  7.55s/it]

Initial portfolio value:100000
Final portfolio value: 2107212.161685642
Final accumulative portfolio value: 21.072121616856418
Maximum DrawDown: -0.770143074534354
Sharpe ratio: 5.909335123505892


100%|██████████| 50/50 [06:15<00:00,  7.52s/it]


In [11]:
print("PERSISTING MODEL.....")
torch.save(model.train_policy.state_dict(), f"models/policy_{experiment_type}.pt")

PERSISTING MODEL.....


In [12]:
print("TESTING.....")
MEIIE_results = {
    "training": train_environment._asset_memory["final"],
    "test": {}
}

TESTING.....


In [13]:
test_environment = MultiPeriodPortfolioOptimizationEnv(
    df_portfolio_test,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=N,
    multi_period_horizon=T,
    features=FEATURES,
    normalize_df=None,
    is_train_mode=False,
    experiment_type=experiment_type
)

In [14]:
policy = MultiPeriodAttentionNetwork(
    num_features=NUM_ASSETS*NUM_FEATURES,
    num_stocks=NUM_ASSETS,
    num_head=4,
    W=N,
    T=T
)

In [15]:
policy.load_state_dict(torch.load(f"models/policy_{experiment_type}.pt"))

<All keys matched successfully>

In [16]:
DRLAgent.DRL_validation(model=model, test_env=test_environment, policy=policy)

Initial portfolio value:100000
Final portfolio value: 528705.7985438819
Final accumulative portfolio value: 5.287057985438818
Maximum DrawDown: -0.3867989334315056
Sharpe ratio: 5.623929839987467


In [17]:
MEIIE_results["test"]["value"] = test_environment._asset_memory["final"]

In [18]:
drl_portfolio_performance = test_environment._asset_memory["final"][1:]
len(drl_portfolio_performance)

70

In [19]:
drl_portfolio_performance = [(x/100000)-1 for x in drl_portfolio_performance]

In [20]:
performance_dataset = pd.read_csv("data/processed/performances_sp100.csv", index_col=0)

In [21]:
performance_dataset["DRL_TRANSFORMER"] = drl_portfolio_performance

In [22]:
performance_dataset.to_csv("data/processed/performances_sp100.csv")